In [1]:
import pandas as pd
import sqlite3

# Reading data from SQL databases

So far we've only talked about reading data from CSV files. That's a pretty common way to store data, but there are many others! Pandas can read from HTML, JSON, SQL, Excel (!!!), HDF5, Stata, and a few other things. In this chapter we'll talk about reading data from SQL databases.

You can read data from a SQL database using the `pd.read_sql` function. `read_sql` will automatically convert SQL column names to DataFrame column names.

`read_sql` takes 2 arguments: a `SELECT` statement, and a database connection object. This is great because it means you can read from *any* kind of SQL database -- it doesn't matter if it's MySQL, SQLite, PostgreSQL, or something else.

This example reads from a SQLite database, but any other database would work the same way.

In [2]:
con = sqlite3.connect("data/weather_2012.sqlite")
df = pd.read_sql("SELECT * from weather_2012 LIMIT 20", con)
df

,id,date_time,temp
0,1,2012-01-01 00:00:00,-1.8
1,2,2012-01-01 01:00:00,-1.8
2,3,2012-01-01 02:00:00,-1.8
3,4,2012-01-01 03:00:00,-1.5
4,5,2012-01-01 04:00:00,-1.5
5,6,2012-01-01 05:00:00,-1.4
6,7,2012-01-01 06:00:00,-1.5
7,8,2012-01-01 07:00:00,-1.4
8,9,2012-01-01 08:00:00,-1.4
9,10,2012-01-01 09:00:00,-1.3


`read_sql` doesn't automatically set the primary key (`id`) to be the index of the dataframe. You can make it do that by adding an `index_col` argument to `read_sql`. 

If you've used `read_csv` a lot, you may have seen that it has an `index_col` argument as well. This one behaves the same way.

In [3]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 10", con, index_col='id')
df

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8
4,2012-01-01 03:00:00,-1.5
5,2012-01-01 04:00:00,-1.5
6,2012-01-01 05:00:00,-1.4
7,2012-01-01 06:00:00,-1.5
8,2012-01-01 07:00:00,-1.4
9,2012-01-01 08:00:00,-1.4


If you want your dataframe to be indexed by more than one column, you can give a list of columns to `index_col`:

In [4]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 10", con, 
                 index_col=['id', 'date_time'])
df

,,temp
id,date_time,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8
4,2012-01-01 03:00:00,-1.5
5,2012-01-01 04:00:00,-1.5
6,2012-01-01 05:00:00,-1.4
7,2012-01-01 06:00:00,-1.5
8,2012-01-01 07:00:00,-1.4
9,2012-01-01 08:00:00,-1.4


#  Writing to a SQLite database

Pandas has a `write_frame` function which creates a database table from a dataframe. Right now this only works for SQLite databases. Let's use it to move our 2012 weather data into SQL.

You'll notice that this function is in `pd.io.sql`. There are a ton of useful functions for reading and writing various kind of data in `pd.io`, and it's worth spending some time exploring them. ([see the documentation!](http://pandas.pydata.org/pandas-docs/stable/io.html))

In [8]:
weather_df = pd.read_csv('data/weather_2012.csv')
con = sqlite3.connect("data/testt_db.sqlite")
con.execute("DROP TABLE IF EXISTS weather_2012")
weather_df.to_sql("weather_20133", con)

We can now read from the `weather_2013` table in  `test_db.sqlite`, and we see that we get the same data back:

In [9]:
con = sqlite3.connect("data/testt_db.sqlite")
df = pd.read_sql("SELECT * from weather_2013 LIMIT 3", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"


The nice thing about having your data in a database is that you can do arbitrary SQL queries. This is cool especially if you're more familiar with SQL. Here's an example of sorting by the Weather column:

In [10]:
con = sqlite3.connect("data/testt_db.sqlite")
df = pd.read_sql("SELECT * from weather_2013 ORDER BY Weather LIMIT 3", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,67,2012-01-03 19:00:00,-16.9,-24.8,50,24,25.0,101.74,Clear
1,114,2012-01-05 18:00:00,-7.1,-14.4,56,11,25.0,100.71,Clear
2,115,2012-01-05 19:00:00,-9.2,-15.4,61,7,25.0,100.80,Clear


If you have a PostgreSQL database or MySQL database, reading from it works exactly the same way as reading from a SQLite database. You create a connection using `psycopg2.connect()` or `MySQLdb.connect()`, and then use

`pd.read_sql("SELECT whatever from your_table", con)`

# Connecting to other kinds of database

To connect to a MySQL database:

*Note: For these to work, you will need a working MySQL / PostgreSQL database, with the correct localhost, database name, etc.*

To connect to a PostgreSQL database:

#### Reshaping Dataframes

In [15]:
import pandas as pd
trials=pd.read_csv('data/trials.csv')

In [16]:
trials

,id,treatment,gender,response
0,1,A,F,5
1,2,A,M,3
2,3,B,F,8
3,4,B,M,9


In [17]:
#reshaping by pivoting
trials.pivot(index='treatment',columns='gender',values='response')

gender,F,M
treatment,,
A,5,3
B,8,9


In [18]:
#pivoting multiple columns
trials.pivot(index='treatment', columns='gender')

id    response   
gender     F  M        F  M
treatment                  
A          1  2        5  3
B          3  4        8  9

In [19]:
#creating multi-level index
trials = trials.set_index(['treatment', 'gender','id'])

In [30]:
trials.sort_values(by=['response'], inplace=True, ascending=False)
trials

response
treatment gender id          
B         M      4          9
          F      3          8
A         F      1          5
          M      2          3

In [8]:
unstacked=trials.unstack(level='gender')
unstacked
#or level=1

id    response   
gender     F  M        F  M
treatment                  
A          1  2        5  3
B          3  4        8  9

In [9]:
# stacking back
stacked=unstacked.stack(level='gender')

In [10]:
stacked

id  response
treatment gender              
A         F        1         5
          M        2         3
B         F        3         8
          M        4         9

In [12]:
#swapping levels
trials_swapped=stacked.swaplevel(0,1)
trials_swapped.groupby('gender').count()

,id,response
gender,,
F,2,2
M,2,2


<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  

In [13]:
#sort rows
sorted_trial=trials_swapped.sort_index()
sorted_trial
# 
sorted_trial.get_level_values(0)

AttributeError: 'DataFrame' object has no attribute 'get_level_values'

In [14]:
#sort rows
sorted_trial=trials_swapped.sort_index(level=0, ascending=False)
sorted_trial

id  response
gender treatment              
M      B           4         9
       A           2         3
F      B           3         8
       A           1         5

#### Aggregate Functions included pivots

In [23]:
agg=sorted_trial.pivot_table(index='treatment',columns='gender',values='response',aggfunc='sum')

In [24]:
agg
#try len, count

gender,F,M
treatment,,
A,5,3
B,8,9
